# Code from GitHub discussion 

https://github.com/pydantic/pydantic/discussions/8592

In [6]:
from dataclasses import dataclass
from typing import Annotated, Any
from typing_extensions import TypeAliasType

from pydantic import PlainValidator, BaseModel, WithJsonSchema

@dataclass
class ThirdParty:
    value: Any

def my_validator(value, info):
    print(f"{info=}")
    return ThirdParty(value)

ThirdPartyType = TypeAliasType(
    'ThirdPartyType',
    Annotated[
        ThirdParty,
        PlainValidator(my_validator),
        WithJsonSchema({"type": "string"})
    ]
)

class MyModel(BaseModel):
    one: ThirdPartyType
    two: ThirdPartyType

mod = MyModel(one=1, two=2, three=3)
print(mod.model_json_schema())
"""
{'$defs': {'ThirdPartyType': {'type': 'int'}},
 'properties': {'one': {'$ref': '#/$defs/ThirdPartyType'},
  'two': {'$ref': '#/$defs/ThirdPartyType'},
 'required': ['one', 'two', 'three'],
 'title': 'MyModel',
 'type': 'object'}
"""

info=ValidationInfo(config={'title': 'MyModel'}, context=None, data={}, field_name='one')
info=ValidationInfo(config={'title': 'MyModel'}, context=None, data={'one': ThirdParty(value=1)}, field_name='one')
{'$defs': {'ThirdPartyType': {'type': 'string'}}, 'properties': {'one': {'$ref': '#/$defs/ThirdPartyType'}, 'two': {'$ref': '#/$defs/ThirdPartyType'}}, 'required': ['one', 'two'], 'title': 'MyModel', 'type': 'object'}


"\n{'$defs': {'ThirdPartyType': {'type': 'int'}},\n 'properties': {'one': {'$ref': '#/$defs/ThirdPartyType'},\n  'two': {'$ref': '#/$defs/ThirdPartyType'},\n 'required': ['one', 'two', 'three'],\n 'title': 'MyModel',\n 'type': 'object'}\n"

# Modified to try adding the JSON schema to the validator separate from the type

In [21]:
from dataclasses import dataclass
from typing import Annotated, Any
from typing_extensions import TypeAliasType

from pydantic import PlainValidator, BaseModel, WithJsonSchema
from pydantic_core import core_schema

@dataclass
class ThirdParty:
    value: Any

def my_validator(value, info):
    print(f"{info=}")
    return ThirdParty(value)


class MyValidator:
    @classmethod
    def __get_pydantic_core_schema__(cls, source_type, handler):
        print(f"In Method {handler.field_name=}")
        def validate(value, info):
            print(f"In validator func {info.field_name=}")
            return str(value)

        return core_schema.chain_schema([
                    core_schema.with_info_plain_validator_function(validate),
                    core_schema.str_schema()
                ]
            )

    @classmethod
    def __get_pydantic_json_schema__(
        cls, _core_schema, handler
    ):
        print("===> In schema method:")
        print(f"\t{_core_schema=}")
        print(f"\t{handler=}")
        print(f"\t{handler(_core_schema)}")
        return handler(core_schema.str_schema())
        



ThirdPartyType = TypeAliasType(
    'ThirdPartyType',
    Annotated[
        ThirdParty,
        MyValidator,
    ]
)

class MyModel(BaseModel):
    one: ThirdPartyType
    two: ThirdPartyType

mod = MyModel(one=1, two=2, three=3)
print(mod.model_json_schema())
"""
{'$defs': {'ThirdPartyType': {'type': 'int'}},
 'properties': {'one': {'$ref': '#/$defs/ThirdPartyType'},
  'two': {'$ref': '#/$defs/ThirdPartyType'},
 'required': ['one', 'two', 'three'],
 'title': 'MyModel',
 'type': 'object'}
""";

In Method handler.field_name='one'
In validator func info.field_name=None
In validator func info.field_name=None
===> In schema method:
	_core_schema={'type': 'chain', 'steps': [{'type': 'function-plain', 'function': {'type': 'with-info', 'function': <function MyValidator.__get_pydantic_core_schema__.<locals>.validate at 0x1211c2020>}}, {'type': 'str'}], 'ref': '__main__.ThirdPartyType:4850290320', 'metadata': {'pydantic_js_annotation_functions': [<bound method MyValidator.__get_pydantic_json_schema__ of <class '__main__.MyValidator'>>], 'pydantic.internal.needs_apply_discriminated_union': False}}
	handler=<pydantic._internal._schema_generation_shared.GenerateJsonSchemaHandler object at 0x120fe79d0>


PydanticInvalidForJsonSchema: Cannot generate a JsonSchema for core_schema.PlainValidatorFunctionSchema ({'type': 'with-info', 'function': <function MyValidator.__get_pydantic_core_schema__.<locals>.validate at 0x1211c2020>})

For further information visit https://errors.pydantic.dev/2.6/u/invalid-for-json-schema